## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [299]:
import pandas as pd

df_original = pd.read_csv('Resources/client_dataset.csv')
df_original.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [300]:
# View the column names in the data
df_original.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [301]:
# Use the describe function to gather some basic statistics
df_original.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00
mean,54837.87,5470190.03,11.36,2022.99,136.27,99.45,5.00,570.26,2.98
std,25487.44,2599806.93,7.02,0.08,183.87,133.16,5.33,18795.52,2.44
min,10033.00,1000886.00,1.00,2022.00,0.01,0.01,0.00,0.00,0.00
25%,33593.00,3196372.00,6.00,2023.00,20.80,14.84,1.45,32.00,1.00
50%,53305.00,5496966.00,11.00,2023.00,68.31,49.89,3.24,68.00,3.00
75%,78498.00,7733869.00,17.00,2023.00,173.16,125.57,6.89,170.00,5.00
max,99984.00,9998480.00,52.00,2023.00,1396.23,846.27,46.43,3958244.00,9.00


In [302]:
# Use this space to do any additional research
# and familiarize yourself with the data.


In [303]:
# What three item categories had the most entries?
top_categories = df_original["category"].value_counts().head(3)
top_categories

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [304]:
# For the category with the most entries, which subcategory had the most entries?
most_entries_category = df_original['category'].value_counts().idxmax()
filtered_df = df_original[df_original['category'] == most_entries_category]
subs = filtered_df['subcategory'].value_counts()
most_sub = subs.idxmax()
most_sub_count = subs.max()
print(f"category: {most_entries_category}\nsub category: {most_sub}\ncount: {most_sub_count}")


category: consumables
sub category: bathroom supplies
count: 6424


In [305]:
# Which five clients had the most entries in the data?
top_five_clients = df_original['client_id'].value_counts().head(5)
top_five_clients

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [306]:
# Store the client ids of those top 5 clients in a list.
top_five_client_ids = top_five_clients.index.tolist()
top_five_client_ids

[33615, 66037, 46820, 38378, 24741]

In [307]:
# How many total units (the qty column) did the client with the most entries order?
client_with_most_entries = df_original['client_id'].value_counts().idxmax()
filtered_df = df_original[df_original['client_id'] == client_with_most_entries]
total_qty_ordered = filtered_df['qty'].sum()
print(f"Client ID with the most entries: {client_with_most_entries}\nTotal units ordered by this client:{total_qty_ordered}", )

Client ID with the most entries: 33615
Total units ordered by this client:64313


## Part 2: Transform the Data
Do we know that this client spent more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [308]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df = df_original.copy()
df['line_subtotal'] = df['unit_price'] * df['qty']
df[['unit_price', 'qty', 'line_subtotal']]

,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28
3,36.42,29,1056.18
4,195.10,20,3902.00
...,...,...,...
54634,83.13,33,2743.29
54635,206.59,47,9709.73
54636,65.66,475,31188.50
54637,1.48,112,165.76


In [309]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
def shipping_price(row):
    weight = row["unit_weight"]
    qty = row["qty"]
    total_weight = weight * qty
    return 7 * total_weight if total_weight > 50 else 10 * total_weight

df['total_weight'] = df['qty'] * df['unit_weight']
df['shipping_price'] = df.apply(shipping_price, axis = 1)
df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']]


,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64
3,36.42,1.23,29,35.67,356.70
4,195.10,46.43,20,928.60,6500.20
...,...,...,...,...,...
54634,83.13,2.25,33,74.25,519.75
54635,206.59,11.70,47,549.90,3849.30
54636,65.66,4.16,475,1976.00,13832.00
54637,1.48,18.04,112,2020.48,14143.36


In [310]:
# Create a column for the line_price using the line_subtotal and the shipping price along with a sales tax of 9.25%
def line_price(row):
    total = row['line_subtotal'] + row['shipping_price']
    return total + (total * 0.0925)

df['line_price'] = df.apply(line_price, axis = 1).round(2)
df[["line_subtotal", "shipping_price", "line_price"]]

,line_subtotal,shipping_price,line_price
0,115164.00,5512.50,131839.08
1,523.95,312.90,914.26
2,527.28,458.64,1077.12
3,1056.18,356.70,1543.57
4,3902.00,6500.20,11364.40
...,...,...,...
54634,2743.29,519.75,3564.87
54635,9709.73,3849.30,14813.24
54636,31188.50,13832.00,49184.90
54637,165.76,14143.36,15632.71


In [311]:
# Create a column for the cost of each line using unit_cost, qty, and
# shipping_price price (assume the shipping cost is exactly what is charged to the client).
def line_cost(row):
    return (row['unit_cost'] * row['qty']) + row['shipping_price']

df['line_cost'] = df.apply(line_cost, axis = 1)
df.head(3)


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18


In [312]:
# Create a column for the profit of each line using line_cost and line_price
def line_profit(row):
    return row['line_price'] - row['line_cost']

df['line_profit'] = df.apply(line_profit, axis = 1)
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,51.60,2.25,33,8,2743.29,74.25,519.75,3564.87,2222.55,1342.32
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,175.46,11.70,47,0,9709.73,549.90,3849.30,14813.24,12095.92,2717.32
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,57.31,4.16,475,6,31188.50,1976.00,13832.00,49184.90,41054.25,8130.65
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,1.22,18.04,112,0,165.76,2020.48,14143.36,15632.71,14280.00,1352.71


## Part 3: Confirm your work
You have email receipts showing the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [313]:
# Check your work using the totals above
order_total_1 = df[df['order_id'] == 2742071]['line_price'].sum()
order_total_2 = df[df['order_id'] == 2173913]['line_price'].sum()
order_total_3 = df[df['order_id'] == 6128929]['line_price'].sum()
print(f"""
    Order ID 2742071 had a total price of ${order_total_1:,.2f}
    Order ID 2173913 had a total price of ${order_total_2:,.2f}
    Order ID 6128929 had a total price of ${order_total_3:,.2f}
    """
)



    Order ID 2742071 had a total price of $152,811.89
    Order ID 2173913 had a total price of $162,388.71
    Order ID 6128929 had a total price of $923,441.25
    


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [314]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_5_clients = df[df['client_id'].isin(top_five_client_ids)]
top_5_clients_spending = top_5_clients.groupby('client_id')['line_price'].sum().reset_index()
top_5_clients_spending = top_5_clients_spending.set_index('client_id').loc[top_five_client_ids].reset_index()
top_5_clients_spending

,client_id,line_price
0,33615,8377308.52
1,66037,10259514.79
2,46820,9743794.36
3,38378,12906550.87
4,24741,82268892.02


In [315]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit.
top_5_clients_data = df[df['client_id'].isin(top_five_client_ids)]
summary_df = top_5_clients_data.groupby('client_id').agg({
    'qty': 'sum',
    'shipping_price': 'sum',
    'line_price': 'sum',
    'line_cost': 'sum',
    'line_profit': 'sum'
}).reset_index()
summary_df = summary_df.set_index('client_id').loc[top_five_client_ids].reset_index()
summary_df.columns = ['client_id', 'qty', 'shipping_price', 'line_price', 'line_cost', 'line_profit']
summary_df

,client_id,qty,shipping_price,line_price,line_cost,line_profit
0,33615,64313,1828984.89,8377308.52,6175313.91,2201994.61
1,66037,43018,1395151.85,10259514.79,7004482.98,3255031.81
2,46820,75768,1601448.84,9743794.36,7007191.64,2736602.72
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
4,24741,239862,5126448.37,82268892.02,45688899.71,36579992.31


In [316]:
# Define the money columns.
money_columns = ['shipping_price', 'line_price', 'line_cost', 'line_profit']
# Define a function that converts a dollar amount to millions.
def convert_to_millions(amount):
    return amount / 1_000_000.0
# Apply the currency_format_millions function to only the money columns.
summary_df_copy = summary_df.copy()
summary_df_copy[money_columns] = summary_df_copy[money_columns].apply(convert_to_millions)
summary_df_copy

# Rename the columns to reflect the change in the money format.
# Format the data and rename the columns to names suitable for presentation.
renamed = summary_df_copy.rename(columns={
    'client_id': 'Client ID',
    'qty': 'Units',
    'shipping_price': 'Shipping (millions)',
    'line_price': 'Total Revenue (millions)',
    'line_cost': 'Total Cost (millions)',
    'line_profit': 'Total Profit (millions)',
})
renamed = renamed.loc[renamed['Client ID'].isin(top_five_client_ids)]
renamed

,Client ID,Units,Shipping (millions),Total Revenue (millions),Total Cost (millions),Total Profit (millions)
0,33615,64313,1.83,8.38,6.18,2.20
1,66037,43018,1.40,10.26,7.00,3.26
2,46820,75768,1.60,9.74,7.01,2.74
3,38378,73667,3.43,12.91,9.63,3.27
4,24741,239862,5.13,82.27,45.69,36.58


In [317]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
sorted = renamed.sort_values(by='Total Profit (millions)', ascending=False)
sorted


,Client ID,Units,Shipping (millions),Total Revenue (millions),Total Cost (millions),Total Profit (millions)
4,24741,239862,5.13,82.27,45.69,36.58
3,38378,73667,3.43,12.91,9.63,3.27
1,66037,43018,1.40,10.26,7.00,3.26
2,46820,75768,1.60,9.74,7.01,2.74
0,33615,64313,1.83,8.38,6.18,2.20
